## 3.2 クラスタリングの量的評価のはなし
このノートでは、クラスタリングのモデル評価について検討しましょう。以下の流れで検討を行います。
1. クラスタリングのタスクの洗い出し
2. 内的妥当性尺度
3. 外的妥当性尺度

### A. クラスタリングタスクの洗い出し
機械学習のモデル評価の一般論として、**モデル評価の指標は解いているタスクによって決まります。**クラスタリングならとりあえずこれを見ておけばよいとか、そういうものではないということです。そこで、クラスタリングが使われる主なタスクを一度洗い出しておきましょう。

1. 探索的データ分析
2. 分類問題を教師なし学習で実現したい場合

探索的データ分析におけるクラスタリングの役割は、機械学習としてのクラスタリングではなく、むしろデータの概要を把握するための分析です。一方で、分類問題を想定しているにも関わらず
* 一部のデータポイントに対してしか、明確にラベルが想定できない。

など、全てのデータポイントにラベルを付与することが困難である場合、クラスタリングによってラベルの予測を実現することが考えられます。

#### A1. 探索的データ分析としてのクラスタリング
この場合に重要なのは、
* 分析者の結果に対する納得感（質的評価）
* 入力空間で仮に可視化できたとき、まとまりのないクラスターが生成されていないか

の2点です。

究極的に重要なのは「分析者の結果に対する納得感」で、例えば顧客のデータであれば実際の顧客の行動とクラスタリングの結果を見ながら、納得できるものかどうか**質的に検討する**ことになります。

また、後者「入力空間で仮に可視化できたとき、まとまりのないクラスターが生成されていないか」ですが、ひとつは**実際に散布図にクラスタリングの結果を落とし込んで確認する**ことが重要です。また、
* 内的妥当性尺度：入力のなんらか性質をどの程度反映しているか。

という観点から、いくつかの数値指標を参考にすることもあります。

#### A2. 分類問題を教師なし学習で実現するためのクラスタリング
この場合に重要なのは、
* 事前にラベルを想定できるようなデータポイントでのクラスタリングの結果の整合性

です。想定される結果を予めいくつかのデータポイントについて準備しておき、クラスタリングの結果との類似度を測るような尺度を**外的妥当性尺度**といいます。

### B. 内的妥当性尺度
入力の何らかの性質をクラスタリングの結果がどれほど反映しているか測る尺度を内的妥当性尺度といいます。内的妥当性尺度には
* k-means法の場合 : silhouette visualizer
* 階層的クラスタリングの場合 : cophenetic相関係数

があります。Pythonの場合、silhouette visualizerは`yellowbrick`パッケージの`cluster`モジュールに計算するための`SilhouetteVisualizer`メソッドがあります。また、cophenetic相関係数は`scipy.hierarchy.cluster`モジュールの`cophenet`メソッドで計算できます。

また、クラスター間で入力変数の分布に差があるかどうかヒストグラムを確認したり、母平均の差の検定を行うこともあります。

#### B1. silhoette visualizer
silhoette visualizerは、縦軸にデータポイント、横軸にsilhoette係数をプロットします。silhoette係数は以下のように定義されます。

**silhoette係数：** データポイント$x_i$と同じクラスター内のその他すべてのデータポイントとの距離の平均値を$a_i$, 最も近くにあるクラスター内のすべてのデータポイントとの距離の平均値を$b_i$とします。このとき、

$$s_i=\frac{b_i-a_i}{\max\{a_i, b_i\}}$$

をデータポイント$x_i$のsilhoette係数といいます。

silhoette係数を確認することで各クラスターが他のクラスターと入力空間上でどれだけ離れているかを確認することが出来ます。

#### B2. cophenetic相関係数
cophenetic相関係数は、データポイント間の入力空間上での距離とデンドログラム上のパス長との相関係数です。

### C. 外的妥当性尺度
クラスタリングの結果が望ましい分類結果とどれだけ類似しているかを測る尺度を外的妥当性尺度といいます。外的妥当性尺度の代表例はRand尺度です。Rand尺度はクラスタリングで目標ているクラスターの定義を思い出すと理解しやすいでしょう。

**Rand尺度：** 全てのデータポイントの組$(x_1,x_2)$に対して、想定解でもクラスターの結果でも同じクラスターに所属しているデータポイントの組の個数を$a$, 想定解でもクラスターの結果でも異なるクラスターに所属しているデータポイントの組の個数を$b$とします。このとき、すべてのデータポイントの組$M=n(n-1)/2$に対して、$RI=(a+b)/M$をRand尺度という。

要するにRand尺度は、すべてのデータポイントの組に対して、以下のようなクロス集計表を書いていることと同じです。
<center>
<img src="rand.png" width = 300px>
</center>
なおこのクロス集計表において、想定解とクラスタリングの結果が独立と仮定したときの$a,b$の期待度数の合計値を$e$とします。このとき、Rand尺度を以下のように修正したものを修正Rand尺度といい、-1以上1以下の値をとり、1に近づくほど良いと評価する方法が良く用いられています。
$$RI_{adj}=\frac{(a+b)-e}{M-e}$$
調整済みRand尺度は`sklearn.metrics`モジュールの`adjusted_rand_score`メソッドで計算できます。